In [1]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

# Neo4j Import of GraphRAG Result Parquet files

This notebook imports the results of the GraphRAG indexing process into the Neo4j Graph database for further processing, analysis or visualization. 

You can also build your own GenAI applications using Neo4j and a number of RAG strategies with LangChain, LlamaIndex, Haystack, and many other frameworks.
See: https://neo4j.com/labs/genai-ecosystem

Here is what the end result looks like:

![](https://dev.assets.neo4j.com/wp-content/uploads/graphrag-neo4j-visualization.png)

## How does it work?

The notebook loads the parquet files from the `output` folder of your indexing process and loads them into Pandas dataframes.
It then uses a batching approach to send a slice of the data into Neo4j to create nodes and relationships and add relevant properties. The id-arrays on most entities are turned into relationships. 

All operations use MERGE, so they are idempotent, and you can run the script multiple times.

If you need to clean out the database, you can run the following statement

```cypher
MATCH (n)
CALL { WITH n DETACH DELETE n } IN TRANSACTIONS OF 25000 ROWS;
```

In [2]:
GRAPHRAG_FOLDER = "C:/Users/TusharJain/ragtest/output"

### Depedendencies

We only need Pandas and the neo4j Python driver with the rust extension for faster network transport.

In [3]:
%pip install --quiet pandas neo4j-rust-ext

Note: you may need to restart the kernel to use updated packages.


In [4]:
import time

import pandas as pd
from neo4j import GraphDatabase

## Neo4j Installation

You can create a free instance of Neo4j [online](https://console.neo4j.io). You get a credentials file that you can use for the connection credentials. You can also get an instance in any of the cloud marketplaces.

If you want to install Neo4j locally either use [Neo4j Desktop](https://neo4j.com/download) or 
the official Docker image: `docker run -e NEO4J_AUTH=neo4j/password -p 7687:7687 -p 7474:7474 neo4j` 

In [5]:
NEO4J_URI = "neo4j+s://f17ab497.databases.neo4j.io"  # or neo4j+s://xxxx.databases.neo4j.io
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "VCTkSYItsCMWq6S_jL6iozFcHEtSkTn7wCRIBpJQNDE"  # your password
NEO4J_DATABASE = "neo4j"

# Create a Neo4j driver
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

## Batched Import

The batched import function takes a Cypher insert statement (needs to use the variable `value` for the row) and a dataframe to import.
It will send by default 1k rows at a time as query parameter to the database to be inserted.

In [6]:
def batched_import(statement, df, batch_size=1000):
    """
    Import a dataframe into Neo4j using a batched approach.

    Parameters: statement is the Cypher query to execute, df is the dataframe to import, and batch_size is the number of rows to import in each batch.
    """
    total = len(df)
    start_s = time.time()
    for start in range(0, total, batch_size):
        batch = df.iloc[start : min(start + batch_size, total)]
        result = driver.execute_query(
            "UNWIND $rows AS value " + statement,
            rows=batch.to_dict("records"),
            database_=NEO4J_DATABASE,
        )
        print(result.summary.counters)
    print(f"{total} rows in {time.time() - start_s} s.")
    return total

## Indexes and Constraints

Indexes in Neo4j are only used to find the starting points for graph queries, e.g. quickly finding two nodes to connect.
Constraints exist to avoid duplicates, we create them mostly on id's of Entity types.

We use some Types as markers with two underscores before and after to distinguish them from the actual entity types.

The default relationship type here is `RELATED` but we could also infer a real relationship-type from the description or the types of the start and end-nodes.

* `__Entity__`
* `__Document__`
* `__Chunk__`
* `__Community__`
* `__Covariate__`

In [7]:
# # create constraints, idempotent operation

# statements = """
# create constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique;
# create constraint document_id if not exists for (d:__Document__) require d.id is unique;
# create constraint entity_id if not exists for (c:__Community__) require c.community is unique;
# create constraint entity_id if not exists for (e:__Entity__) require e.id is unique;
# create constraint entity_title if not exists for (e:__Entity__) require e.name is unique;
# create constraint entity_title if not exists for (e:__Covariate__) require e.title is unique;
# create constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique;
# """.split(";")

# for statement in statements:
#     if len((statement or "").strip()) > 0:
#         print(statement)
#         driver.execute_query(statement)

## Import Process

### Importing the Documents

We're loading the parquet file for the documents and create nodes with their ids and add the title property.
We don't need to store text_unit_ids as we can create the relationships and the text content is also contained in the chunks.

In [8]:
doc_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_documents.parquet", columns=["id", "title"]
)
doc_df.head(2)

,id,title
0,675b5991cecfe09978375ab0bd2c6cc1,Class export.txt
1,59e7890c3a3585b04faee5f1547f0b2e,function compilation.txt


In [9]:
# Import documents
statement = """
MERGE (d:__Document__ {id:value.id})
SET d += value {.title}
"""

batched_import(statement, doc_df)

{'_contains_updates': True, 'properties_set': 3}
3 rows in 1.311204433441162 s.


3

### Loading Text Units

We load the text units, create a node per id and set the text and number of tokens.
Then we connect them to the documents that we created before.

In [10]:
text_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_text_units.parquet",
    columns=["id", "text", "n_tokens", "document_ids"],
)
text_df.head(2)

,id,text,n_tokens,document_ids
0,9fe05946567c481b265abefd72089ed1,**********************************************...,1200,[0d1514a1958b087cee042478fb199791]
1,4831fc8424e707db3a38d845acdf498c,_hotspot_click FOR EVENT hotspot_click\n ...,1200,[0d1514a1958b087cee042478fb199791]


In [11]:
statement = """
MERGE (c:__Chunk__ {id:value.id})
SET c += value {.text, .n_tokens}
WITH c, value
UNWIND value.document_ids AS document
MATCH (d:__Document__ {id:document})
MERGE (c)-[:PART_OF]->(d)
"""

batched_import(statement, text_df)

{'_contains_updates': True, 'properties_set': 1212}
606 rows in 1.3756356239318848 s.


606

### Loading Nodes

For the nodes we store id, name, description, embedding (if available), human readable id.

In [12]:
entity_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_entities.parquet",
    columns=[
        "name",
        "type",
        "description",
        "human_readable_id",
        "id",
        "description_embedding",
        "text_unit_ids",
    ],
)
entity_df.head(2)

,name,type,description,human_readable_id,id,description_embedding,text_unit_ids
0,/CGDC/LCLRQMU04,INCLUDE,Include statement for CLRQ_PROCESS functionality,0,d69b4aa7f613480fbea468bd689bffeb,"[0.01129966415464878, 0.0364234521985054, 0.04...",[9fe05946567c481b265abefd72089ed1]
1,/CGDC/LCLRQMU05,INCLUDE,Include statement for CLRQ_RTBC_PROCESS functi...,1,9f54a217d9974b9cad80538f9db53731,"[0.013413391076028347, 0.023699110373854637, 0...",[9fe05946567c481b265abefd72089ed1]


In [13]:
entity_statement = """
MERGE (e:__Entity__ {id:value.id})
SET e += value {.human_readable_id, .description, name:replace(value.name,'"','')}
WITH e, value
CALL db.create.setNodeVectorProperty(e, "description_embedding", value.description_embedding)
CALL apoc.create.addLabels(e, case when coalesce(value.type,"") = "" then [] else [apoc.text.upperCamelCase(replace(value.type,'"',''))] end) yield node
UNWIND value.text_unit_ids AS text_unit
MATCH (c:__Chunk__ {id:text_unit})
MERGE (c)-[:HAS_ENTITY]->(e)
"""

batched_import(entity_statement, entity_df)

{'_contains_updates': True, 'properties_set': 3000}
{'_contains_updates': True, 'properties_set': 3000}


ConstraintError: {code: Neo.ClientError.Schema.ConstraintValidationFailed} {message: Node(2802) already exists with label `__Entity__` and property `name` = '/CGDC/CLRQ_SAVE'}

In [86]:
# Drop duplicates based on 'name' and 'type' columns, keeping the first occurrence
entity_df = entity_df.drop_duplicates(subset=['name', 'type'], keep='first')
print(entity_df)

                               name      type  \
0             /CGDC/CL_CLRQ_PROCESS     CLASS   
1                    GTT_CR_DOCTYPE      TYPE   
2                  GTY_CF_POP_ENDAT      TYPE   
3                  GTT_CF_POP_ENDAT      TYPE   
4                      GTT_CLRQ_ERR      TYPE   
..                              ...       ...   
165  /CGDC/CLRQ_TRGR_REAL_TIME_PRCS  FUNCTION   
166                        LT_TABLE  VARIABLE   
167                            COEP     TABLE   
168                            COBK     TABLE   
169                           COEPR     TABLE   

                                           description  human_readable_id  \
0    The "/CGDC/CL_CLRQ_PROCESS" is an ABAP class s...                  0   
1    Standard table type for document types in the ...                  1   
2    Structure type defining fields for performance...                  2   
3    Standard table type of gty_cf_pop_endat for st...                  3   
4    Standard table type f

In [87]:
# Filter the entity_df to find rows where the name column contains '/CGDC/_CLRQHD'
filtered_df = entity_df[entity_df['name'].str.contains('/CGDC/_CLRQHD')]
print(filtered_df)


            name   type                                        description  \
5  /CGDC/_CLRQHD  TABLE  The entity "/CGDC/_CLRQHD" refers to a custom ...   

   human_readable_id                                id  \
5                176  e56afc402f2b4587862c794a193e84b9   

                               description_embedding  \
5  [-0.016181832179427147, 0.04186629503965378, 0...   

                                       text_unit_ids  
5  [1590fdd44a1ddf4fedc40fe600d8ebeb, 360d31cc8ed...  


In [88]:
# Check for duplicates in entity_df based on the 'id' column
duplicates = entity_df[entity_df.duplicated(subset='name', keep=False)]
print(duplicates)

Empty DataFrame
Columns: [name, type, description, human_readable_id, id, description_embedding, text_unit_ids]
Index: []


### Import Relationships

For the relationships we find the source and target node by name, using the base `__Entity__` type.
After creating the `RELATED` relationships, we set the description as attribute.

In [89]:
rel_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_relationships.parquet",
    columns=[
        "source",
        "target",
        "id",
        "rank",
        "weight",
        "human_readable_id",
        "description",
        "text_unit_ids",
    ],
)
rel_df.head(2)

,source,target,id,rank,weight,human_readable_id,description,text_unit_ids
0,/CGDC/CL_CLRQ_PROCESS,GTT_CR_DOCTYPE,2e972777cbd843d481fb120205e5ec42,50,10.0,0,The gtt_cr_doctype type is defined within the ...,[77afae11468def2f00d30a57c55c02c8]
1,/CGDC/CL_CLRQ_PROCESS,GC_ITM_BILL_STATUS,2cc08790c74c4eaf945b974b06ad579e,53,10.0,1,The gc_itm_bill_status constants are defined w...,[77afae11468def2f00d30a57c55c02c8]


In [90]:
rel_statement = """
    MATCH (source:__Entity__ {name:replace(value.source,'"','')})
    MATCH (target:__Entity__ {name:replace(value.target,'"','')})
    // not necessary to merge on id as there is only one relationship per pair
    MERGE (source)-[rel:RELATED {id: value.id}]->(target)
    SET rel += value {.rank, .weight, .human_readable_id, .description, .text_unit_ids}
    RETURN count(*) as createdRels
"""

batched_import(rel_statement, rel_df)

{'_contains_updates': True, 'relationships_created': 548, 'properties_set': 3288}
548 rows in 0.7544255256652832 s.


548

### Importing Communities

For communities we import their id, title, level.
We connect the `__Community__` nodes to the start and end nodes of the relationships they refer to.

Connecting them to the chunks they orignate from is optional, as the entites are already connected to the chunks.

In [91]:
community_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_communities.parquet",
    columns=["id", "level", "title", "text_unit_ids", "relationship_ids"],
)

community_df.head(2)

,id,level,title,text_unit_ids,relationship_ids
0,3,0,Community 3,"[02518262df9188eb68cefe5a80524002,0a9879312a25...","[2e972777cbd843d481fb120205e5ec42, 2cc08790c74..."
1,16,0,Community 16,"[05155c2d98a9cff7f7f3d2dacc83583a,66ce6976b36a...","[cd8c4e7b89fe45ce944a46c404d03ad6, 756b2e05f60..."


In [92]:
statement = """
MERGE (c:__Community__ {community:value.id})
SET c += value {.level, .title}
/*
UNWIND value.text_unit_ids as text_unit_id
MATCH (t:__Chunk__ {id:text_unit_id})
MERGE (c)-[:HAS_CHUNK]->(t)
WITH distinct c, value
*/
WITH *
UNWIND value.relationship_ids as rel_id
MATCH (start:__Entity__)-[:RELATED {id:rel_id}]->(end:__Entity__)
MERGE (start)-[:IN_COMMUNITY]->(c)
MERGE (end)-[:IN_COMMUNITY]->(c)
RETURn count(distinct c) as createdCommunities
"""

batched_import(statement, community_df)

{'_contains_updates': True, 'labels_added': 91, 'relationships_created': 1301, 'nodes_created': 91, 'properties_set': 273}
91 rows in 0.6258261203765869 s.


91

### Importing Community Reports

Fo the community reports we create nodes for each communitiy set the id, community, level, title, summary, rank, and rank_explanation and connect them to the entities they are about.
For the findings we create the findings in context of the communities.

In [93]:
community_report_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_community_reports.parquet",
    columns=[
        "id",
        "community",
        "level",
        "title",
        "summary",
        "findings",
        "rank",
        "rank_explanation",
        "full_content",
    ],
)
community_report_df.head(2)

,id,community,level,title,summary,findings,rank,rank_explanation,full_content
0,a926b1ad-cdcd-4ad3-8a03-5c919763d773,79,2,SAP ABAP /CGDC/CL_CLRQ_PROCESS Class Ecosystem,This report delves into the /CGDC/CL_CLRQ_PROC...,[{'explanation': 'The /CGDC/CL_CLRQ_PROCESS cl...,9.5,The rating reflects the high relevance of this...,# SAP ABAP /CGDC/CL_CLRQ_PROCESS Class Ecosyst...
1,88908cb7-fdff-4561-8ad7-9f4bfc586043,80,2,RTB_DOC_USAGE_CHECK and its Role in SAP ABAP D...,The RTB_DOC_USAGE_CHECK method is a specialize...,[{'explanation': 'RTB_DOC_USAGE_CHECK is a piv...,9.0,This report is highly relevant to software dev...,# RTB_DOC_USAGE_CHECK and its Role in SAP ABAP...


In [94]:
# Import communities
community_statement = """
MERGE (c:__Community__ {community:value.community})
SET c += value {.level, .title, .rank, .rank_explanation, .full_content, .summary}
WITH c, value
UNWIND range(0, size(value.findings)-1) AS finding_idx
WITH c, value, finding_idx, value.findings[finding_idx] as finding
MERGE (c)-[:HAS_FINDING]->(f:Finding {id:finding_idx})
SET f += finding
"""
batched_import(community_statement, community_report_df)

{'_contains_updates': True, 'labels_added': 441, 'relationships_created': 441, 'nodes_created': 441, 'properties_set': 1869}
91 rows in 0.5125257968902588 s.


91

### Importing Covariates

Covariates are for instance claims on entities, we connect them to the chunks where they originate from.

In [95]:
cov_df = pd.read_parquet(f"{GRAPHRAG_FOLDER}/create_final_covariates.parquet")
#                         columns=["id","text_unit_id"])
cov_df.head(2)
# Subject id do not match entity ids

,id,human_readable_id,covariate_type,type,description,subject_id,subject_type,object_id,object_type,status,start_date,end_date,source_text,text_unit_id,document_ids,n_tokens
0,9bd73361-5e94-4867-9c7c-3e4319486f85,1,claim,None,None,No claims or facts relevant to information dis...,None,None,None,None,None,None,None,77afae11468def2f00d30a57c55c02c8,[675b5991cecfe09978375ab0bd2c6cc1],1200
1,12a69a30-ae08-4d24-9148-9f9de96cfca8,2,claim,None,None,,None,None,None,None,None,None,None,29a7062d6a5b159a1c53444e91625410,[675b5991cecfe09978375ab0bd2c6cc1],1200


In [96]:
# Import covariates
cov_statement = """
MERGE (c:__Covariate__ {id:value.id})
SET c += apoc.map.clean(value, ["text_unit_id", "document_ids", "n_tokens"], [NULL, ""])
WITH c, value
MATCH (ch:__Chunk__ {id: value.text_unit_id})
MERGE (ch)-[:HAS_COVARIATE]->(c)
"""
batched_import(cov_statement, cov_df)

{'_contains_updates': True, 'labels_added': 52, 'relationships_created': 52, 'nodes_created': 52, 'properties_set': 267}
52 rows in 0.20680952072143555 s.


52

### Visualize your data

You can now [Open] Neo4j on Aura, you need to log in with either SSO or your credentials.

Or open https://workspace-preview.neo4j.io and connect to your local instance, remember the URI is `neo4j://localhost` and `neo4j` as username and `password` as password.

In "Explore" you can explore by using visual graph patterns and then explore and expand further.

In "Query", you can open the left sidebar and explore by clicking on the nodes and relationships.
You can also use the co-pilot to generate Cypher queries for your, here are some examples.

#### Show a few `__Entity__` nodes and their relationships (Entity Graph)

```cypher
MATCH path = (:__Entity__)-[:RELATED]->(:__Entity__)
RETURN path LIMIT 200
```

#### Show the Chunks and the Document (Lexical Graph)

```cypher
MATCH (d:__Document__) WITH d LIMIT 1
MATCH path = (d)<-[:PART_OF]-(c:__Chunk__)
RETURN path LIMIT 100
```

####  Show a Community and it's Entities

```cypher
MATCH (c:__Community__) WITH c LIMIT 1
MATCH path = (c)<-[:IN_COMMUNITY]-()-[:RELATED]-(:__Entity__)
RETURN path LIMIT 100
```

#### Show everything

```cypher
MATCH (d:__Document__) WITH d LIMIT 1
MATCH path = (d)<-[:PART_OF]-(:__Chunk__)-[:HAS_ENTIY]->()-[:RELATED]-()-[:IN_COMMUNITY]->()
RETURN path LIMIT 250
```

We showed the visualization of this last query at the beginning.

If you have questions, feel free to reach out in the GraphRAG discord server: 
https://discord.gg/graphrag